In [ ]:
from pandas import read_csv

filename = './dataset/cost_of_living/cost_of_living_per_country_in_dollars.csv'
df = read_csv(filename)

df

In [ ]:
from pandas import read_csv
filename = 'dataset//earnings_split/EAR_4MMN_CUR_NB_A-filtered-2021-03-21.csv'
df_incomes_columns_csv = ["ref_area.label","classif1.label","time","obs_value"]
df_incomes_columns_names = ["Country","Currency","Year","Income"]
df_incomes = read_csv(filename,usecols=df_incomes_columns_csv)
df_incomes = df_incomes[(df_incomes["classif1.label"]=="Currency: U.S. dollars") & (df_incomes["time"]>2009)].drop("classif1.label", axis=1)

df_incomes = df_incomes.rename(columns={k:v for k,v in zip(df_incomes_columns_csv,df_incomes_columns_names)})
df_incomes.head()

In [ ]:
from pandas import merge as pd_merge
filename = 'dataset//earnings_split/EAR_4MTH_SEX_ECO_CUR_NB_A-filtered-2021-03-21.csv'
df_field_incomes_columns_csv = ["ref_area.label","classif1.label","classif2.label","time","obs_value"]
df_field_incomes_columns_names = ["Country","Field","Currency","Year","Income"]
df_field_incomes = read_csv(filename,usecols=df_field_incomes_columns_csv)
df_field_incomes = df_field_incomes[(df_field_incomes["classif2.label"]=="Currency: U.S. dollars") & (df_field_incomes["time"]>2009)].drop("classif2.label", axis=1)

df_field_incomes = df_field_incomes.rename(columns={k:v for k,v in zip(df_field_incomes_columns_csv,df_field_incomes_columns_names)})
df_avg_income = df_field_incomes[df_field_incomes["Field"]=="Economic activity (Aggregate): Total"].drop("Field", axis=1).rename(columns={"Income":"Average Income"})
df_field_incomes["Field"] = list(map(lambda text: text.split(":")[1][1:],df_field_incomes["Field"]))
df_field_incomes = pd_merge(df_field_incomes,df_avg_income, on=["Year", "Country"],how="inner")
df_field_incomes

In [ ]:
import string
fields = [letter + '.' for letter in string.ascii_uppercase[:19]] + ['Total']
fields = tuple(fields)

df_field_incomes = df_field_incomes.loc[df_field_incomes['Field'].str.startswith(fields)]
df_field_incomes = df_field_incomes.drop_duplicates()
df_field_incomes = df_field_incomes.drop('Average Income', axis=1)
df_field_incomes

In [ ]:
df_merged = pd_merge(df, df_field_incomes, on=['Country', 'Year'], how='inner')
df_merged.head(20)

In [ ]:
def calculate_wealth(row):
    return row['Income'] / row['Cost of living']

df_merged['Wealth Factor'] = df_merged.apply(calculate_wealth, axis=1)
df_merged

In [ ]:
filename = f'./dataset/wealth_factor/wealth.csv'
with open(filename, 'w') as file:
    df_merged.to_csv(file, index=False, line_terminator='\n')